In [4]:
import requests 
from requests import RequestException
import re
import json

In [24]:
def get_page(url):
    try:
        headers = {'accept': "application/json, text/plain, */*",
                    'accept-encoding': "gzip, deflate, br",
                    'accept-language': "en,zh-CN;q=0.9,zh;q=0.8,zh-TW;q=0.7",
                    'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36",
                    'cookie' : '_uab_collina=155209551008218263663793; upload_channel=qiniu; ws_address=wss%3A//ws.zsxq.com/ws%3Fversion%3Dv1.10%26access_token%3D7354A7B5-BD0F-FC6B-77F7-DF55D4DAEEBF; user_id=48285181142148; name=superadmin; avatar_url=https%3A//images.zsxq.com/FkAlF_oJR7I509FrUt5O0m25QYbf%3Fe%3D1906272000%26token%3DkIxbL07-8jAj8w1n4s9zv64FuZZNEATmlU_Vm6zD%3Aznj5c7Tb2f1F4Yu9P8Bk7FDSQCc%3D; sensorsdata2015jssdkcross=%7B%22distinct_id%22%3A%2216a260d16fae9-0907998bfde48b-7a296633-190296-16a260d16fb6a8%22%2C%22%24device_id%22%3A%2216a260d16fae9-0907998bfde48b-7a296633-190296-16a260d16fb6a8%22%2C%22props%22%3A%7B%22%24latest_traffic_source_type%22%3A%22%E7%9B%B4%E6%8E%A5%E6%B5%81%E9%87%8F%22%2C%22%24latest_referrer%22%3A%22%22%2C%22%24latest_referrer_host%22%3A%22%22%2C%22%24latest_search_keyword%22%3A%22%E6%9C%AA%E5%8F%96%E5%88%B0%E5%80%BC_%E7%9B%B4%E6%8E%A5%E6%89%93%E5%BC%80%22%7D%7D; m_target=https://wx.zsxq.com/mweb/views/joingroup/join_group.html?group_id=518858821154; zsxq_access_token=7354A7B5-BD0F-FC6B-77F7-DF55D4DAEEBF'}
        response = requests.get(url,headers=headers)
        if response.status_code == 200:
            return response
        return None
    except RequestException:
        return None

In [28]:
import datetime
from urllib import parse
def parse_content(temp_content):
    try:
        out_temp = temp_content.get('topic').get('talk')
        if 'images' in out_temp.keys():
            li = out_temp.get('images')
            b = out_temp.get('text')
            a = []
            for i in li:
                a.append(i.get('large').get('url'))    
            return a, b

        else:
#             return re.findall(r'.*?type=ha.*?>(.*?).*?',out_temp.get('text'),re.S)[0] 
            return out_temp.get('text')
    except Exception as e:
        print('error')
    
def parse_json(json):
    tmp_content = json.get("resp_data").get("topics")
#     pattern_content = re.compile(r'{'owner'.*?<e type.*?/>(.*?)')
    for element_content in tmp_content:
        temp_dict = {}
        user_name = element_content.get('topic').get('talk').get('owner').get('name')
        raw_time = element_content.get('topic').get('create_time')[:-9].replace('T',' ')
        content = parse_content(element_content)
        
        temp_dict['name'] = user_name
        temp_dict['time'] = raw_time
        temp_dict['content'] = content
        yield temp_dict

def write_to_file(x):
    with open('out.csv','a',encoding='utf-8') as f:
        f.write(json.dumps(x,ensure_ascii=False)+'\n')
        
def main():
    pass

first = get_page('https://api.zsxq.com/v1.10/search/all?count=10&scope=joined&keyword=2019%E5%AE%9E%E6%88%98%E7%AC%AC%E4%BA%8C%E6%9C%9F').json()
for i in parse_json(first):
    write_to_file(i)
    
offset = [11,41,71,101,131,161]
for i in offset:
    raw_dict = get_page(f'https://api.zsxq.com/v1.10/search/all?count=10&scope=joined&index={i}&keyword=2019%E7%AC%AC%E4%BA%8C%E6%9C%9F%E5%AE%9E%E6%88%98').json()
# topics = raw_dict.get("resp_data").get("topics")
    for j in parse_json(raw_dict):
        write_to_file(j)